In [1]:
%matplotlib widget
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib import colors, gridspec
import seaborn as sns
import visualize as vis

import preprocessing as pp
import calculate as cc
from read_WDF_class import WDF


folder = "/data/2021-10-03-EcoleVerreEtDiffusion/"
filename = "exampleM4.wdf"
filepath = os.path.join(folder, filename)

In [2]:
fff = WDF(filepath, verbose=True)

Reading the file: "exampleM4.wdf"


================================= Block : WDF1 =================================
size: 512, offset: 0
WdfFlag--------------------------------- : 	WdfXYXY
PointsPerSpectrum----------------------- : 	1015
Capacity-------------------------------- : 	46440
Count----------------------------------- : 	46440
AccumulationCount----------------------- : 	1
YlistLength----------------------------- : 	1
XlistLength----------------------------- : 	1015
DataOriginCount------------------------- : 	5
ApplicationName------------------------- : 	WiRE
ApplicationVersion---------------------- : 	5.1.0 build 8867
ScanType-------------------------------- : 	StreamLine
MeasurementType------------------------- : 	Map
StartTime------------------------------- : 	Thu Oct 10 15:58:30 2019
EndTime--------------------------------- : 	Fri Oct 11 11:39:48 2019
SpectralUnits--------------------------- : 	Counts
LaserWaveLength------------------------- : 	514.38
Title----------------

### Here are some of the treatments you might apply to your input spectra:
Do this first:
```python
fff = pp.order(fff)
```
Normalizing:
```python
fff = pp.normalize(fff)
```
Selecting the zone of interest:
```python
fff = pp.select_zone(fff, left=125, right=None)
```
Removing the Cosmic Rays:
```python
fff = pp.remove_CRs(fff)
```
Removing the baseline:
(don't do this on the cloud!)
```python
baseline = cc.baseline_als(fff.spectra, p=0.001)
```
Clean the spectra with the PCA:
```python
fff = pp.pca_clean(fff, n_components=21)
```

In [45]:
sns.set_style("dark")
show1 = vis.ShowSelected(fff)
show1.fig.set_size_inches(8, 6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
fff = pp.order(fff)


In [4]:
# =============================================================================
#                                     PCA...
# =============================================================================
fff = pp.pca_clean(fff, n_components=21)

In [5]:
fff = pp.normalize(fff, method="area")
mix, components, reconstucted_spectra = cc.deconvolute_nmf(fff, 3, max_iter=133)

norma = np.linalg.norm(components, axis=-1, keepdims=True)
norma = np.trapz(components, x=fff.x_values, axis=-1)[:, None]
components /= norma
mix *= norma.T

/home/dejan/anaconda3/envs/Raman/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 133 reached. Increase it to improve convergence.
  warnings.warn(


In [6]:
_n_components = len(components)

spectra_reconstructed = np.dot(mix, components)
_mix_reshaped = mix.reshape(fff.n_y, fff.n_x, _n_components)

col_norm = colors.Normalize(vmin=0, vmax=_n_components)
color_set = ScalarMappable(norm=col_norm, cmap="brg")

# infer the number of subplots and their disposition from n_components
fi, _ax = plt.subplots(int(np.floor(np.sqrt(_n_components))),
                       int(np.ceil(_n_components /
                                   np.floor(np.sqrt(_n_components))
                                   )))
if _n_components > 1:
    _ax = _ax.ravel()
else:
    _ax = [_ax]
for _i in range(_n_components):
    _ax[_i].plot(fff.x_values, components[_i].T, color=color_set.to_rgba(_i))
    _ax[_i].set_title(f'Component {_i}')
    _ax[_i].set_yticks([])
try:
    fi.text(0.5, 0.04,
            f"{params['XlistDataType']} recordings"
            f"in {params['XlistDataUnits']} units",
            ha='center')
except:
    pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# %%
# =============================================================================
#                       Plotting the main plot...(heatmap)
# =============================================================================
_start_pos = 0
_n_fig_rows = int(np.floor(np.sqrt(_n_components)))
_n_fig_cols = int(np.ceil(_n_components / np.floor(np.sqrt(_n_components))))

fig = plt.figure()
gs = gridspec.GridSpec(_n_fig_rows+1, _n_fig_cols, left=0.05, right=0.98, wspace=0.05)
_ax=[]
for i in range(_n_fig_rows):
    for j in range(_n_fig_cols):
        _ax.append(fig.add_subplot(gs[i, j]))
if _n_components <= 1:
    _ax = [_ax]
ax_spectra = fig.add_subplot(gs[_n_fig_rows,:])
spectra_flag = False
def onclick(event):
    '''Double-clicking on a pixel will show the (cleaned) spectrum
    corresponding to that pixel, as well as its deconvolution on the components
    and again the reconstruction for visual comparison'''
    global ax_spectra, spectra_flag, sp_l, rs_l, c_l
    if event.inaxes:
        x_pos = round(event.xdata)
        y_pos = round(event.ydata)
        broj = round(y_pos*fff.n_x + x_pos)
        spec_num = np.round(y_pos*fff.n_x - _start_pos + x_pos)
        if spectra_flag:
            if event.button!=1:
                sp_l.set_ydata(fff.spectra[spec_num])
                sp_l.set_label(f'(cleaned) spectrum n°{broj}')
                rs_l.set_ydata(spectra_reconstructed[broj])
                for k in range(_n_components):
                    c_l[k].set_ydata(components[k]*mix[broj][k])
                    c_l[k].set_label(f'C {k} = {100*mix[broj][k]/(np.sum(mix[broj])):.1f}%)')
 
                ax_spectra.relim()
                ax_spectra.legend()

        elif event.button!=1:
            spectra_flag = True
            sp_l, = ax_spectra.plot(fff.x_values, fff.spectra[spec_num], 'o', alpha=0.3,
                       label=f'(cleaned) spectrum n°{broj}')
            rs_l, = ax_spectra.plot(fff.x_values, spectra_reconstructed[broj], '--k',
                    label='reconstructed spectrum')
            c_l = []
            for k in range(_n_components):
                c_l.append(*ax_spectra.plot(fff.x_values, components[k]*mix[broj][k],
                        color=color_set.to_rgba(k),
                        label=f'C {k} = {100*mix[broj][k]/(np.sum(mix[broj])):.1f}%)'))


            ax_spectra.set_title(f'deconvolution of the spectrum from: '
                         f'line {y_pos} & column {x_pos}')
            ax_spectra.legend()
            fig.show()
    else:
        pass

for _i in range(_n_components):
    sns.heatmap(_mix_reshaped[:, :, _i], ax=_ax[_i], cmap="jet", annot=False)
#    _ax[_i].set_aspect(_s_y/_s_x)
    _ax[_i].set_title(f'Component {_i}', color=color_set.to_rgba(_i),
                      fontweight='extra bold')
fig.suptitle('Heatmaps showing the abundance of individual components'
             ' throughout the scanned area.');
fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

9